<a href="https://colab.research.google.com/github/nickwinters1/DS-Unit-4-Sprint-1-NLP/blob/master/NickWintersKaggleAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "nickwinters1" 
os.environ['KAGGLE_KEY'] = "b45322b5d97e37bf445d2d8c978b40d2" # key from the json file
!kaggle competitions download -c ds3-which-whisky

  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 39.0MB/s]
  0% 0.00/1.91k [00:00<?, ?B/s]
100% 1.91k/1.91k [00:00<00:00, 1.97MB/s]
  0% 0.00/463k [00:00<?, ?B/s]
100% 463k/463k [00:00<00:00, 63.9MB/s]


In [2]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
import pandas as pd

In [0]:
train=pd.read_csv('train.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2874 entries, 0 to 2873
Data columns (total 7 columns):
id              2874 non-null int64
author          2874 non-null object
description     2874 non-null object
price           2811 non-null float64
ratingValue     2874 non-null int64
pert_alcohol    2814 non-null float64
category        2586 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 157.2+ KB


In [6]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.5,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.9,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.0,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.5,1.0
4,6,Davin de Kergommeaux,"After 40 years in barrels, the trademark Canad...",199.0,96,45.0,NaN


In [7]:
train.describe(include='number')

,id,price,ratingValue,pert_alcohol,category
count,2874.000000,2811.000000,2874.000000,2814.000000,2586.000000
mean,2075.814544,225.297937,86.361517,48.043019,1.637664
std,1177.805945,990.619608,4.511554,6.298527,0.963049
min,1.000000,7.500000,60.000000,33.000000,1.000000
25%,1087.250000,50.000000,84.000000,43.300000,1.000000
50%,2109.500000,80.000000,87.000000,46.000000,1.000000
75%,3102.000000,140.000000,90.000000,51.375000,2.000000
max,4157.000000,26650.000000,97.000000,98.600000,4.000000


In [8]:
train.describe(exclude='number')

,author,description
count,2874,2874
unique,12,2871
top,John Hansell,Swiss-based Chapter 7 released this 19 year ol...
freq,992,2


In [0]:
train['category']=train['category'].dropna()

In [0]:
train=train.dropna()

In [11]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.21.2)


In [0]:
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


In [0]:
 vect=TfidfVectorizer(stop_words='english')
pipeline = Pipeline([
    ('vect',vect),
    ('clf', SGDClassifier())
])

In [0]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__max_iter': (10, 50, 80),
}

In [0]:
features = ['id', 'author', 'description', 'price', 'ratingValue', 'pert_alcohol']
target = 'category'

In [16]:
grid_search = GridSearchCV(pipeline, parameters, cv=5,
                          n_jobs=-1,verbose=1)



grid_search.fit(train['description'],train[target])
print('best score: %0.3f'% grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  1.7min finished


best score: 0.945
Best parameters set:
	clf__alpha: 1e-05
	clf__max_iter: 80
	clf__penalty: 'elasticnet'
	vect__max_df: 0.75
	vect__ngram_range: (1, 2)


In [0]:
test=pd.read_csv('test.csv')

In [0]:
predict=grid_search.predict(test['description'])

In [19]:
test['category']=predict
test.head()

,id,author,description,price,ratingValue,pert_alcohol,category
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0,2.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3,2.0
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0,4.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8,1.0
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9,1.0


In [20]:
submission=test.drop(columns=['author','description','price','ratingValue','pert_alcohol'])
submission['category']=submission['category'].astype('int')
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,4
3,1024,1
4,1902,1


In [0]:
submission.to_csv('submission.csv',index=False)

In [24]:
!kaggle competitions submit -c ds3-which-whisky -f submission.csv -m "Submission 1"

100% 1.91k/1.91k [00:02<00:00, 882B/s]
Successfully submitted to DS3 Which Whisky

In [0]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500,
                   algorithm='randomized',
                   n_iter=10)

In [0]:


lsi = Pipeline([('vect', vect),('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', SGDClassifier())])

parameters = {
    'lsi__vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'lsi__vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__max_iter': (10, 50, 80),
}




In [28]:
grid_search = GridSearchCV(pipe, parameters, cv=3,
                          n_jobs=-1,verbose=10)



grid_search.fit(train['description'],train[target])
print('best score: %0.3f'% grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   56.2s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 

best score: 0.929
Best parameters set:
	clf__alpha: 1e-06
	clf__max_iter: 20
	clf__penalty: 'l2'
	lsi__vect__max_df: 1.0
	lsi__vect__ngram_range: (1, 2)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [0]:
predict=grid_search.predict(test['description'])
test['category']=predict
submission=test.drop(columns=['author','description','price','ratingValue','pert_alcohol'])
submission['category']=submission['category'].astype('int')
submission.to_csv('submission1.csv',index=False)

In [0]:
import spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 1.2MB/s 
y
  Stored in directory: /tmp/pip-ephem-wheel-cache-n7u9c7cx/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp=en_core_web_lg.load()

In [0]:
def word_vector(docs):
  return[nlp(doc).vector for doc in docs]
 

In [0]:
x=word_vector(train['description'])


In [0]:
train['vector']=x

In [0]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category,vector
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0,"[-0.02859438, 0.20648584, -0.08717509, -0.1194..."
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0,"[-0.010728728, 0.22426474, -0.030888626, -0.06..."
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0,"[-0.0014092241, 0.17896245, -0.06267574, -0.05..."
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0,"[-0.038889475, 0.20561711, -0.10034756, -0.071..."
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0,"[-0.10290412, 0.25401792, -0.08515398, -0.0901..."


In [0]:
sgdc=SGDClassifier()
sgdc.fit(x,train['category'])

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:

predict=sgdc.predict(word_vector(test['description']))
test['category']=predict
submission=test.drop(columns=['author','description','price','ratingValue','pert_alcohol'])
submission['category']=submission['category'].astype('int')
submission.to_csv('submission2.csv',index=False)
#80% ish